In [1]:
from langchain_core.messages import SystemMessage,HumanMessage
from typing_extensions import TypedDict
from typing import Annotated,List
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langchain_core.messages import AnyMessage

c:\Users\madda\Desktop\Udemy\udemy-langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatOllama(model = "llama3.2")
response = llm.invoke("Hi")
response

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-10-02T16:38:57.2486094Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2139335500, 'load_duration': 272267600, 'prompt_eval_count': 26, 'prompt_eval_duration': 875309400, 'eval_count': 8, 'eval_duration': 990196000, 'model_name': 'llama3.2'}, id='run--c41e3d9e-05b4-4bc3-bece-ee528217b856-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [3]:
# defining tools

def add(a:int, b:int) -> int:
    """Adds 2 integers and returns an integer
    args: 2 integer values
    output: addition of given 2 integer numbers
    """
    return a+b

def sub(a:int, b:int) -> int:
    """Subtracts 2 integers and returns an integer
    args: 2 integer values
    output: subtraction of given 2 integer numbers
    """
    return a-b

def multiply(a:int, b:int) -> int:
    """Multiplies 2 integers and returns an integer
    args: 2 integer values
    output: mulitplication of given 2 integer numbers
    """
    return a*b

def divide(a:int, b:int) -> float:
    """Divides 2 integers and returns a float value
    args: 2 integer values
    output: division of given 2 integer numbers
    """
    return a/b


In [4]:
tools = [add,sub,multiply,divide]

llm_with_tools = llm.bind_tools(tools)

In [34]:
llm_with_tools.invoke("addition of 2 and 3 is:")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-10-02T16:24:57.9155608Z', 'done': True, 'done_reason': 'stop', 'total_duration': 43667139400, 'load_duration': 177429800, 'prompt_eval_count': 416, 'prompt_eval_duration': 39618792600, 'eval_count': 22, 'eval_duration': 3869698700, 'model_name': 'llama3.2'}, id='run--ba3c6515-3a31-407a-8fb6-4e56240c7729-0', tool_calls=[{'name': 'add', 'args': {'a': '2', 'b': '3'}, 'id': 'f003f3f3-058c-42c9-8d55-f06bef1e6757', 'type': 'tool_call'}], usage_metadata={'input_tokens': 416, 'output_tokens': 22, 'total_tokens': 438})

In [5]:
class State(TypedDict):
    messages:Annotated[List[AnyMessage],add_messages]

In [6]:
def llm_call(state:State):
    return {'messages':[llm_with_tools.invoke(state['messages'])]}

In [7]:
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode, tools_condition

builder = StateGraph(State)

builder.add_node("llm_call",llm_call)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"llm_call")
builder.add_conditional_edges('llm_call',tools_condition)
builder.add_edge('tools','llm_call')
builder.add_edge('llm_call',END)

memory = MemorySaver()
builder_graph = builder.compile(checkpointer=memory, interrupt_before=["llm_call"])

In [8]:
config = {'configurable':{'thread_id':'12'}}

In [11]:
from pprint import pprint

response = builder_graph.invoke({"messages":"addition of 2 and 2 is?"},config=config)

In [12]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

addition of 2 and 2 is?
================================ Human Message =================================

addition of 2 and 2 is?


In [13]:
builder_graph.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='addition of 2 and 2 is?', additional_kwargs={}, response_metadata={}, id='6e1c9846-ca94-41d3-8432-6a30a25eaf86'), HumanMessage(content='addition of 2 and 2 is?', additional_kwargs={}, response_metadata={}, id='3aa91a78-3dd5-447a-aed9-fb17f94f921b')]}, next=('llm_call',), config={'configurable': {'thread_id': '12', 'checkpoint_ns': '', 'checkpoint_id': '1f09fae5-1956-6c77-8002-035403adafc6'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '12'}, created_at='2025-10-02T16:39:09.035839+00:00', parent_config={'configurable': {'thread_id': '12', 'checkpoint_ns': '', 'checkpoint_id': '1f09fae5-1948-6e05-8001-1e2c8242c696'}}, tasks=(PregelTask(id='69042cdb-1c43-53f2-f6db-c224d12b4b3f', name='llm_call', path=('__pregel_pull', 'llm_call'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [ ]:
for m in builder_graph.stream(None, stream_mode='values',config=config):
    pprint(m)

{'messages': [HumanMessage(content='addition of 2 and 2 is?', additional_kwargs={}, response_metadata={}, id='6e1c9846-ca94-41d3-8432-6a30a25eaf86'),
              HumanMessage(content='addition of 2 and 2 is?', additional_kwargs={}, response_metadata={}, id='3aa91a78-3dd5-447a-aed9-fb17f94f921b')]}
